In [1]:
import  os, glob
import  random, csv
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import pickle
import cv2
from PIL import Image
import adapt
from adapt.feature_based import DANN
from tensorflow import keras
from keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Reshape
from tensorflow.keras.optimizers import Adam
from keras.utils.np_utils import to_categorical

# The path of the datasets, use dict format
dataset_path = {"base": "dataset/ccpd/splitted_plates_base", 
                "challenge":"dataset/ccpd/splitted_plates_challenge",
               "db":"dataset/ccpd/splitted_plates_db",
               "fn":"dataset/ccpd/splitted_plates_fn",
               "weather":"dataset/ccpd/splitted_plates_weather"}
save_check_pt = './checkpoints_DANN'

def load_csv(root, filename, name2label):
    # From csv file return images dir,labels list
    if not os.path.exists(os.path.join(root, filename)):
        images = []
        for name in name2label.keys(): 
            images += glob.glob(os.path.join(root, name, '*.png'))
            images += glob.glob(os.path.join(root, name, '*.jpg'))
            images += glob.glob(os.path.join(root, name, '*.jpeg'))
        #print(len(images), images)
        random.shuffle(images) # shuffle images
        with open(os.path.join(root, filename), mode='w', newline='') as f:
            writer = csv.writer(f)
            for img in images:  
                name = img.split(os.sep)[-2]
                label = name2label[name]
                writer.writerow([img, label])
            print('written into csv file:', filename)
    # read existed csv
    images, labels = [], []
    with open(os.path.join(root, filename)) as f:
        reader = csv.reader(f)
        for row in reader:
            img, label = row
            label = int(label)
            images.append(img)
            labels.append(label) 
    # return img dir and label
    return images, labels

def load_ccpd(root, mode='train'):
    name2label = {}  # "sq...":0
    # iterate sub dir, sort, while keep mapping
    for name in sorted(os.listdir(os.path.join(root))):
        # skip non file folder
        if not os.path.isdir(os.path.join(root, name)):
            continue
        # give each category a number
        name2label[name] = len(name2label.keys())
    images, labels = load_csv(root, 'images.csv', name2label)
    if mode == 'train':  # 20%
        images = images[:int(0.2 * len(images))]
        labels = labels[:int(0.2 * len(labels))]
    elif mode == 'val':  # 20% = 60%->80%
        images = images[int(0.6 * len(images)):int(0.8 * len(images))]
        labels = labels[int(0.6 * len(labels)):int(0.8 * len(labels))]
    else:  # all
        images = images
        labels = labels
    return images, labels, name2label

# def readCcpdImg(images_dir):
#     X = []
#     for img_dir in images_dir:
#         img = cv2.imread(img_dir)
#         img = cv2.resize(img,(32,32))
#         X.append(img)
#     X = np.array(X)
#     return X

def readCcpdImg(images_dir):
    X = []
    for img_dir in images_dir:
        img = Image.open(img_dir)
        img = img.convert('L') # conver to grayscale images
        img = img.resize([32, 16])
        img_np = np.asarray(img)
        X.append(img_np.reshape([-1]))
    X = np.array(X)
    return X

def get_img_label(path, mode,num_classes=35):
    images_dir, labels, table = load_ccpd(dataset_path[path],mode=mode)
    images = readCcpdImg(images_dir)
    labels = np.array(labels)
    labels = to_categorical(labels, num_classes=num_classes)
    return images,labels

def get_encoder(input_shape=(512,)):
    model = Sequential()
    model.add(Dense(512, activation='relu',
                    input_shape=input_shape))
    model.add(Dense(256, activation="relu"))
    model.add(Dense(128, activation="sigmoid"))
    return model

def get_task(input_shape=(128,)):
    model = Sequential()
    model.add(Dense(128, activation='relu',input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(34, activation="softmax"))
    return model

def get_discriminator(input_shape=(128,)):
    model = Sequential()
    model.add(Dense(128, activation='relu',input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

base_images,base_labels = get_img_label('base', 'all', num_classes=34)
weather_images,weather_labels = get_img_label('weather', 'all', num_classes=34)
weather_images_t,weather_labels_t = get_img_label('weather', 'train', num_classes=34)
challenge_images,challenge_labels = get_img_label('challenge', 'all', num_classes=34)
challenge_images_t,challenge_labels_t = get_img_label('challenge', 'train', num_classes=34)
db_images,db_labels = get_img_label('db', 'all', num_classes=34)
db_images_t,db_labels_t = get_img_label('db', 'train', num_classes=34)
fn_images,fn_labels = get_img_label('fn', 'all', num_classes=34)
fn_images_t,fn_labels_t = get_img_label('fn', 'train', num_classes=34)
print("load data finished!")

load data finished!


In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
print(np.shape(base_images))
print(np.shape(base_labels))
print(np.shape(weather_images_t))
print(np.shape(weather_labels_t))
weather_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
weather_model.fit(base_images, base_labels,weather_images_t,weather_labels_t, epochs=70,verbose=1,batch_size = 32)
weather_model.score(base_images, base_labels)
weather_model.score(weather_images, weather_labels)
weather_model.save_weights(save_check_pt+ '/weather')

(8446, 512)
(8446, 34)
(762, 512)
(762, 34)
Epoch 1/70
286/286 [==============================] - 2s 3ms/step - loss: 2.6231 - acc: 0.3274 - disc_loss: 1.3715 - disc_acc: 0.5563
Epoch 2/70
286/286 [==============================] - 1s 3ms/step - loss: 1.9272 - acc: 0.5332 - disc_loss: 1.3644 - disc_acc: 0.5626
Epoch 3/70
286/286 [==============================] - 1s 3ms/step - loss: 1.5195 - acc: 0.6095 - disc_loss: 1.3516 - disc_acc: 0.5796
Epoch 4/70
286/286 [==============================] - 1s 3ms/step - loss: 1.2671 - acc: 0.6598 - disc_loss: 1.3459 - disc_acc: 0.5823
Epoch 5/70
286/286 [==============================] - 1s 3ms/step - loss: 1.0807 - acc: 0.7079 - disc_loss: 1.3410 - disc_acc: 0.5844
Epoch 6/70
286/286 [==============================] - 1s 3ms/step - loss: 0.9062 - acc: 0.7602 - disc_loss: 1.3359 - disc_acc: 0.5915
Epoch 7/70
286/286 [==============================] - 1s 3ms/step - loss: 0.7775 - acc: 0.8010 - disc_loss: 1.3236 - disc_acc: 0.5981
Epoch 8/70
286/286

286/286 [==============================] - 1s 3ms/step - loss: 0.0733 - acc: 0.9817 - disc_loss: 1.3280 - disc_acc: 0.5930
Epoch 62/70
286/286 [==============================] - 1s 3ms/step - loss: 0.0630 - acc: 0.9850 - disc_loss: 1.3174 - disc_acc: 0.6000
Epoch 63/70
286/286 [==============================] - 1s 3ms/step - loss: 0.0742 - acc: 0.9795 - disc_loss: 1.3255 - disc_acc: 0.5922
Epoch 64/70
286/286 [==============================] - 1s 3ms/step - loss: 0.0948 - acc: 0.9731 - disc_loss: 1.3207 - disc_acc: 0.5943
Epoch 65/70
286/286 [==============================] - 1s 3ms/step - loss: 0.0529 - acc: 0.9885 - disc_loss: 1.3072 - disc_acc: 0.6072
Epoch 66/70
286/286 [==============================] - 1s 3ms/step - loss: 0.0609 - acc: 0.9835 - disc_loss: 1.3187 - disc_acc: 0.5995
Epoch 67/70
286/286 [==============================] - 1s 4ms/step - loss: 0.0397 - acc: 0.9919 - disc_loss: 1.3151 - disc_acc: 0.6046
Epoch 68/70
286/286 [==============================] - 1s 3ms/step 

In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
print(np.shape(base_images))
print(np.shape(base_labels))
print(np.shape(challenge_images_t))
print(np.shape(challenge_labels_t))
challenge_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
challenge_model.fit(base_images, base_labels,challenge_images_t,challenge_labels_t, epochs=87,verbose=1,batch_size = 32)
challenge_model.score(base_images, base_labels)
challenge_model.score(challenge_images, challenge_labels)
challenge_model.save_weights(save_check_pt+ '/challenge')

(8446, 512)
(8446, 34)
(109, 512)
(109, 34)
Epoch 1/87
266/266 [==============================] - 2s 4ms/step - loss: 2.6318 - acc: 0.3350 - disc_loss: 1.3893 - disc_acc: 0.5326
Epoch 2/87
266/266 [==============================] - 1s 4ms/step - loss: 1.9398 - acc: 0.5163 - disc_loss: 1.3575 - disc_acc: 0.5721
Epoch 3/87
266/266 [==============================] - 1s 3ms/step - loss: 1.4797 - acc: 0.6336 - disc_loss: 1.3432 - disc_acc: 0.5884
Epoch 4/87
266/266 [==============================] - 1s 3ms/step - loss: 1.1908 - acc: 0.6871 - disc_loss: 1.3284 - disc_acc: 0.5943
Epoch 5/87
266/266 [==============================] - 1s 3ms/step - loss: 1.0104 - acc: 0.7265 - disc_loss: 1.3140 - disc_acc: 0.6115
Epoch 6/87
266/266 [==============================] - 1s 3ms/step - loss: 0.8728 - acc: 0.7678 - disc_loss: 1.2979 - disc_acc: 0.6163
Epoch 7/87
266/266 [==============================] - 1s 4ms/step - loss: 0.7677 - acc: 0.7937 - disc_loss: 1.2940 - disc_acc: 0.6202
Epoch 8/87
266/266

266/266 [==============================] - 1s 3ms/step - loss: 0.0788 - acc: 0.9802 - disc_loss: 1.2349 - disc_acc: 0.6505
Epoch 62/87
266/266 [==============================] - 1s 3ms/step - loss: 0.0662 - acc: 0.9832 - disc_loss: 1.2598 - disc_acc: 0.6326
Epoch 63/87
266/266 [==============================] - 1s 3ms/step - loss: 0.0681 - acc: 0.9834 - disc_loss: 1.2360 - disc_acc: 0.6409
Epoch 64/87
266/266 [==============================] - 1s 3ms/step - loss: 0.0580 - acc: 0.9864 - disc_loss: 1.2568 - disc_acc: 0.6235
Epoch 65/87
266/266 [==============================] - 1s 3ms/step - loss: 0.0688 - acc: 0.9832 - disc_loss: 1.2398 - disc_acc: 0.6394
Epoch 66/87
266/266 [==============================] - 1s 3ms/step - loss: 0.0675 - acc: 0.9825 - disc_loss: 1.2439 - disc_acc: 0.6405
Epoch 67/87
266/266 [==============================] - 1s 3ms/step - loss: 0.0773 - acc: 0.9780 - disc_loss: 1.2329 - disc_acc: 0.6504
Epoch 68/87
266/266 [==============================] - 1s 3ms/step 

In [4]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
print(np.shape(base_images))
print(np.shape(base_labels))
print(np.shape(db_images_t))
print(np.shape(db_labels_t))
db_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
db_model.fit(base_images, base_labels,db_images_t,db_labels_t, epochs=88,verbose=1,batch_size = 32)
db_model.score(base_images, base_labels)
db_model.score(db_images, db_labels)
db_model.save_weights(save_check_pt+ '/db')

(8446, 512)
(8446, 34)
(373, 512)
(373, 34)
Epoch 1/88
269/269 [==============================] - 2s 4ms/step - loss: 2.6060 - acc: 0.3292 - disc_loss: 1.3746 - disc_acc: 0.5535
Epoch 2/88
269/269 [==============================] - 1s 4ms/step - loss: 1.9326 - acc: 0.5337 - disc_loss: 1.3459 - disc_acc: 0.5806
Epoch 3/88
269/269 [==============================] - 1s 4ms/step - loss: 1.4952 - acc: 0.6280 - disc_loss: 1.3183 - disc_acc: 0.6167
Epoch 4/88
269/269 [==============================] - 1s 3ms/step - loss: 1.2306 - acc: 0.6769 - disc_loss: 1.3383 - disc_acc: 0.5783
Epoch 5/88
269/269 [==============================] - 1s 3ms/step - loss: 1.0684 - acc: 0.7100 - disc_loss: 1.2981 - disc_acc: 0.6220
Epoch 6/88
269/269 [==============================] - 1s 3ms/step - loss: 0.8885 - acc: 0.7554 - disc_loss: 1.3029 - disc_acc: 0.6116
Epoch 7/88
269/269 [==============================] - 1s 4ms/step - loss: 0.7642 - acc: 0.8007 - disc_loss: 1.2995 - disc_acc: 0.6235
Epoch 8/88
269/269

269/269 [==============================] - 1s 3ms/step - loss: 0.0545 - acc: 0.9864 - disc_loss: 1.3196 - disc_acc: 0.5964
Epoch 62/88
269/269 [==============================] - 1s 3ms/step - loss: 0.0779 - acc: 0.9788 - disc_loss: 1.3193 - disc_acc: 0.5961
Epoch 63/88
269/269 [==============================] - 1s 3ms/step - loss: 0.0842 - acc: 0.9762 - disc_loss: 1.3161 - disc_acc: 0.6095
Epoch 64/88
269/269 [==============================] - 1s 3ms/step - loss: 0.0515 - acc: 0.9878 - disc_loss: 1.3068 - disc_acc: 0.6068
Epoch 65/88
269/269 [==============================] - 1s 3ms/step - loss: 0.0533 - acc: 0.9873 - disc_loss: 1.3219 - disc_acc: 0.5972
Epoch 66/88
269/269 [==============================] - 1s 3ms/step - loss: 0.0688 - acc: 0.9801 - disc_loss: 1.3161 - disc_acc: 0.6016
Epoch 67/88
269/269 [==============================] - 1s 3ms/step - loss: 0.0514 - acc: 0.9872 - disc_loss: 1.3209 - disc_acc: 0.6005
Epoch 68/88
269/269 [==============================] - 1s 3ms/step 

In [5]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
print(np.shape(base_images))
print(np.shape(base_labels))
print(np.shape(fn_images_t))
print(np.shape(fn_labels_t))
fn_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
fn_model.fit(base_images, base_labels,fn_images_t,fn_labels_t, epochs=87,verbose=1,batch_size = 32)
fn_model.score(base_images, base_labels)
fn_model.score(fn_images, fn_labels)
fn_model.save_weights(save_check_pt+ '/fn')

(8446, 512)
(8446, 34)
(48, 512)
(48, 34)
Epoch 1/87
264/264 [==============================] - 2s 5ms/step - loss: 2.5996 - acc: 0.3267 - disc_loss: 1.3878 - disc_acc: 0.5377
Epoch 2/87
264/264 [==============================] - 1s 4ms/step - loss: 1.8889 - acc: 0.5469 - disc_loss: 1.3367 - disc_acc: 0.5966
Epoch 3/87
264/264 [==============================] - 1s 3ms/step - loss: 1.4694 - acc: 0.6390 - disc_loss: 1.3185 - disc_acc: 0.6108
Epoch 4/87
264/264 [==============================] - 1s 3ms/step - loss: 1.1984 - acc: 0.6877 - disc_loss: 1.3036 - disc_acc: 0.6173
Epoch 5/87
264/264 [==============================] - 1s 3ms/step - loss: 0.9998 - acc: 0.7350 - disc_loss: 1.2526 - disc_acc: 0.6481
Epoch 6/87
264/264 [==============================] - 1s 3ms/step - loss: 0.8570 - acc: 0.7798 - disc_loss: 1.2471 - disc_acc: 0.6544
Epoch 7/87
264/264 [==============================] - 1s 3ms/step - loss: 0.7293 - acc: 0.8087 - disc_loss: 1.2175 - disc_acc: 0.6672
Epoch 8/87
264/264 [

264/264 [==============================] - 1s 3ms/step - loss: 0.0792 - acc: 0.9790 - disc_loss: 1.2103 - disc_acc: 0.6700
Epoch 62/87
264/264 [==============================] - 1s 3ms/step - loss: 0.0845 - acc: 0.9786 - disc_loss: 1.2084 - disc_acc: 0.6600
Epoch 63/87
264/264 [==============================] - 1s 3ms/step - loss: 0.0902 - acc: 0.9742 - disc_loss: 1.2026 - disc_acc: 0.6669
Epoch 64/87
264/264 [==============================] - 1s 3ms/step - loss: 0.0827 - acc: 0.9783 - disc_loss: 1.2217 - disc_acc: 0.6551
Epoch 65/87
264/264 [==============================] - 1s 3ms/step - loss: 0.0666 - acc: 0.9825 - disc_loss: 1.2081 - disc_acc: 0.6590
Epoch 66/87
264/264 [==============================] - 1s 3ms/step - loss: 0.0699 - acc: 0.9825 - disc_loss: 1.1961 - disc_acc: 0.6775
Epoch 67/87
264/264 [==============================] - 1s 4ms/step - loss: 0.0726 - acc: 0.9798 - disc_loss: 1.1759 - disc_acc: 0.6705
Epoch 68/87
264/264 [==============================] - 1s 3ms/step 